파인튜닝
```
LLM을 추가학습
소량의 고품질데이터로 모델을 재조정해서 직업별 성능 향상
```
파인튜닝vs프롬프트엔지니어링
```
프롬프트엔지니어링
    모델의 가중치를 변경하지않고 입력프롬프트를 조정해 원하는 출력을 유도
```
파인튜닝
```
    모델의 가중치를 업데이트, 특정 데이터셋에 최적화
    종류:
        Full File-Tuning : 모델의 모든 가중치를 튜닝
        PEFT(Parameter-Effective Fine-Tuning): 소수의 파라메터만 업데이트, 효율성 극대화
            LoRA, QLoRA
```
LoRA(Low-Rank Adaptation), QLoRA(Quantized LoRA)
```
    LoRA: 모델의 가중치 행렬에 소규모 추가 파라메터(저랭크 행렬)를 삽입하여 학습
    원본 모델은 유지, 어뎁터만 저장/배포
    Llama-3를 한국어 데이터로 LoRA 파인튜어 
    
    QLoRA: LoRA에 4비트 또는 8비트 양자를 적용
    메모리 사용량을 절감
```
한국어 LLM 파인튜닝
```
영어중심의 LLM 한국어 데이터가 부족
토크나이져가 한국어에 최적화 안됨
해결 : 한국어 특화모델(KoGPT2. Llama-3-Open-Ko, Mistal-Ko)
    한국어 데이터셋 이용(KoAlpaca, NIA한국어 대회 데이터셋)
    토크나이져 재학습, 한국어 전용 토크나이져져
```

In [4]:
!pip install transformers

In [5]:
!conda list pytorch

# packages in environment at /opt/miniconda3/envs/llm_env:
#
# Name                    Version                   Build  Channel
pytorch                   2.5.1           gpu_mps_py39hbd1d365_204  


In [2]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00


In [7]:
# pytorch cpu(전용) 2.0.1
# conda install pytorch=2.0.1 torchvision torchaudio cpuonly -c pytorch

# Transformers는 호환되는 버전으로
# conda install transformers==4.31.0

#conda uninstall pillow -y

#pip install hf_xet

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Lima -3 --> 경량화 모델
model_name = 'facebook/opt-125m'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
#pip install peft
#LoRA 설정 정의
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r = 8, # lora가 추가하는 저랭크 행렬의 차원 # W = W + W'
    lora_alpha=16,
    target_modules=['q_proj','v_proj'], #query, value projection에만 lora 적용
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM" #gpt류 모델에서 텍스트 생성과제 지정
)
# LoRA 모델 적용
model = get_peft_model(model,lora_config)
model.print_trainable_parameters() # trainable parameter 수 확인

ImportError: dlopen(/opt/miniconda3/envs/llm_env/lib/python3.10/site-packages/torch/_C.cpython-310-darwin.so, 0x0002): Symbol not found: __ZN4absl12lts_2025012712log_internal10LogMessagelsIiTnNSt3__19enable_ifIXntsr4absl16HasAbslStringifyIT_EE5valueEiE4typeELi0EEERS2_RKS6_
  Referenced from: <F0CE594F-5059-3403-BEDE-CC2EF3170AD7> /opt/miniconda3/envs/llm_env/lib/libprotobuf.29.3.0.dylib
  Expected in:     <621B4947-F73F-3962-8DDB-2484D6B77411> /opt/miniconda3/envs/llm_env/lib/libabsl_log_internal_message.2501.0.0.dylib

QLoRA LoRA 구조에 4비트 양자화를 추가, 메모리 사용량 줄임

In [4]:
from transformers import BitsAndBytesConfig
import torch
# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4', #실수형 4비트로 정규화
    bnb_4bit_compute_dtype = torch.bfloat16 # 연사용 데이터 타입
)
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_model = 'auto'
)
# 메모리 사용량 확인

print(f'memory: {model.get_momory_footprint()/le9:.2f}GB')

ImportError: dlopen(/opt/miniconda3/envs/llm_env/lib/python3.10/site-packages/torch/_C.cpython-310-darwin.so, 0x0002): Symbol not found: __ZN4absl12lts_2025012712log_internal10LogMessagelsIiTnNSt3__19enable_ifIXntsr4absl16HasAbslStringifyIT_EE5valueEiE4typeELi0EEERS2_RKS6_
  Referenced from: <F0CE594F-5059-3403-BEDE-CC2EF3170AD7> /opt/miniconda3/envs/llm_env/lib/libprotobuf.29.3.0.dylib
  Expected in:     <621B4947-F73F-3962-8DDB-2484D6B77411> /opt/miniconda3/envs/llm_env/lib/libabsl_log_internal_message.2501.0.0.dylib